In [ ]:
import os
from PIL import Image
import numpy as np
from rembg import remove
import cv2

def resize_image(img, size=(225, 225)):
    return img.resize(size, Image.Resampling.LANCZOS)

def remove_background_batch(img_list):
    print("Removing background for batch...")
    return [remove_small_objects(np.array(remove(np.array(img)))) for img in img_list]

def remove_small_objects(img_np, min_size=500):
    gray = cv2.cvtColor(img_np, cv2.COLOR_RGB2GRAY)
    _, binary = cv2.threshold(gray, 1, 255, cv2.THRESH_BINARY)

    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    mask = np.zeros_like(binary)

    for contour in contours:
        area = cv2.contourArea(contour)
        if area >= min_size:
            cv2.drawContours(mask, [contour], -1, 255, thickness=cv2.FILLED)

    result = cv2.bitwise_and(img_np, img_np, mask=mask)
    return result

def add_black_background(img):
    img = Image.fromarray(img).convert("RGBA")
    background = Image.new("RGBA", img.size, "black")
    img_with_bg = Image.alpha_composite(background, img)
    img_with_bg = img_with_bg.convert("RGB")
    return img_with_bg

def find_extreme_points(img, background_color=(0, 0, 0), tolerance=3):
    img_np = np.array(img)

    lower_bound = np.array(background_color) - tolerance
    upper_bound = np.array(background_color) + tolerance
    mask = ~np.all((img_np >= lower_bound) & (img_np <= upper_bound), axis=-1)

    coords = np.argwhere(mask)
    if coords.size == 0:
        return (0, 0), img.size
    top_left = coords.min(axis=0)
    bottom_right = coords.max(axis=0)

    return top_left, bottom_right

def pad_extreme_points(img, size=(225, 225), background_color=(0, 0, 0)):
    top_left, bottom_right = find_extreme_points(img, background_color)
    img_cropped = img.crop((*top_left[::-1], *bottom_right[::-1]))

    ratio = min(size[0] / img_cropped.width, size[1] / img_cropped.height)
    new_size = (int(img_cropped.width * ratio), int(img_cropped.height * ratio))
    img_resized = img_cropped.resize(new_size, Image.Resampling.LANCZOS)

    new_img = Image.new("RGB", size, background_color)
    new_img.paste(img_resized, ((size[0] - new_size[0]) // 2, (size[1] - new_size[1]) // 2))

    return new_img

input_directories = [
    r"C:\Users\USER\Desktop\Selected Perogram\Data set\Scyphiphora hydrophyllacea",
    r"C:\Users\USER\Desktop\Selected Perogram\Data set\Avicennia alba",
    r"C:\Users\USER\Desktop\Selected Perogram\Data set\Bruguiera cylindrica",
    r"C:\Users\USER\Desktop\Selected Perogram\Data set\Scaevola taccada",
]

output_folder = r"C:\Users\USER\Desktop\Selected Perogram\Data set 4 Jenis"
os.makedirs(output_folder, exist_ok=True)

for input_subfolder in input_directories:
    folder_name = os.path.basename(input_subfolder)
    output_subfolder = os.path.join(output_folder, folder_name)
    os.makedirs(output_subfolder, exist_ok=True)

    img_list = []
    img_paths = []

    print(f"Processing folder: {input_subfolder}")

    for filename in os.listdir(input_subfolder):
        if filename.lower().endswith(('.jpeg', '.jpg', '.png')):
            img_path = os.path.join(input_subfolder, filename)
            img = Image.open(img_path)
            img_resized = resize_image(img)
            img_list.append(img_resized)
            img_paths.append((img_path, filename))

    print(f"Total images to process: {len(img_list)}")

    batch_size = 20
    file_counter = 1
    for batch_start in range(0, len(img_list), batch_size):
        batch_end = batch_start + batch_size
        batch = img_list[batch_start:batch_end]
        batch_paths = img_paths[batch_start:batch_end]

        print(f"Processing batch {batch_start // batch_size + 1}: {len(batch)} images")

        no_bg_images = remove_background_batch(batch)

        for i, (img_no_bg, (img_path, filename)) in enumerate(zip(no_bg_images, batch_paths)):
            print(f"Processing image {batch_start + i + 1}/{len(img_list)}: {filename}")

            img_with_bg = add_black_background(img_no_bg)

            final_img = pad_extreme_points(img_with_bg, size=(225, 225), background_color=(0, 0, 0))

            output_path = os.path.join(output_subfolder, f"{folder_name.replace(' ', '.').replace('-', '_')}_{file_counter}.jpg")
            final_img.save(output_path)
            print(f"Processed and saved: {output_path}")
            file_counter += 1


Rotasi 4 Sudut

In [ ]:
import os
from PIL import Image

# Direktori input dan output
data_folders = [
    r"C:\Users\USER\Desktop\Selected Perogram\Data set\Data Uji Asli\Avicennia alba",
   r"C:\Users\USER\Desktop\Selected Perogram\Data set\Data Uji Asli\Bruguiera cylindrica",
    r"C:\Users\USER\Desktop\Selected Perogram\Data set\Data Uji Asli\Scaevola taccada",
    r"C:\Users\USER\Desktop\Selected Perogram\Data set\Data Uji Asli\Scyphiphora hydrophyllacea",
]
output_folder = r"C:\Users\USER\Desktop\Selected Perogram\Data set\Data set 4 Jenis"

# Rotasi yang akan dilakukan
rotations = [0, 90, 270]  # Tidak termasuk 180 untuk menghindari redundansi

# Membuat folder output jika belum ada
os.makedirs(output_folder, exist_ok=True)

# Proses gambar
def process_images(data_folders, output_folder, rotations):
    for folder in data_folders:
        folder_name = os.path.basename(folder)  # Menggunakan nama folder asli
        folder_output_path = os.path.join(output_folder, folder_name)
        os.makedirs(folder_output_path, exist_ok=True)  # Membuat subfolder berdasarkan nama folder input

        image_counter = 1  # Counter untuk penamaan file mulai dari 1

        for file_name in os.listdir(folder):
            file_path = os.path.join(folder, file_name)

            if file_name.lower().endswith(('png', 'jpg', 'jpeg')):
                try:
                    # Membuka gambar
                    with Image.open(file_path) as img:
                        augmentations = {
                            "original": img,
                            "horizontal": img.transpose(Image.FLIP_LEFT_RIGHT),
                            "vertical": img.transpose(Image.FLIP_TOP_BOTTOM)
                        }

                        for aug_name, augmented_img in augmentations.items():
                            for angle in rotations:
                                # Merotasi gambar
                                rotated_img = augmented_img.rotate(angle, expand=True)
                                
                                # Membuat nama file output dengan penomoran berdasarkan folder
                                output_file_name = f"{folder_name.replace(' ', '_')}{image_counter}_{aug_name}_{angle}.jpg"
                                output_path = os.path.join(folder_output_path, output_file_name)
                                
                                # Menyimpan gambar yang sudah dirotasi
                                rotated_img.save(output_path)
                                print(f"Saved: {output_path}")

                        image_counter += 1  # Increment counter setelah semua augmentasi selesai
                except Exception as e:
                    print(f"Error processing {file_path}: {e}")

process_images(data_folders, output_folder, rotations)


Preproses B


In [ ]:
import cv2
import numpy as np
import glob
import os

# Folder input dan output
input_paths = [
    r"C:\Users\USER\Desktop\Selected Perogram\Data set 4 Jenis\Avicennia alba",
    r"C:\Users\USER\Desktop\Selected Perogram\Data set 4 Jenis\Bruguiera cylindrica",
    r"C:\Users\USER\Desktop\Selected Perogram\Data set 4 Jenis\Scaevola taccada",
    r"C:\Users\USER\Desktop\Selected Perogram\Data set 4 Jenis\Scyphiphora hydrophyllacea",
]

output_base_path = r"C:\Users\USER\Desktop\Selected Perogram\Preproses\Tahap B"

# Proses setiap folder
for input_path in input_paths:
    folder_name = os.path.basename(input_path)
    output_path = os.path.join(output_base_path, folder_name)

    if not os.path.exists(output_path):
        os.makedirs(output_path)

    images = glob.glob(os.path.join(input_path, '*.jpg'))

    for img_file in images:
        img = cv2.imread(img_file)
        if img is None:
            print(f"Failed to read image: {img_file}")
            continue


        # Grayscale Conversion
        img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # CLAHE
        clahe = cv2.createCLAHE(clipLimit=5.0, tileGridSize=(8, 8))
        img_clahe = clahe.apply(img_gray)


        # Dilation
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
        img_dilated = cv2.dilate(img_clahe, kernel, iterations=2)

        # Save processed image
        output_file = os.path.join(output_path, os.path.basename(img_file))
        cv2.imwrite(output_file, img_dilated)
        print(f"Processed image saved at: {output_file}")


Otsu

In [ ]:
import os
import cv2
import numpy as np
import time


def otsu_threshold_and_largest_object(input_folder, binary_output_folder, grayscale_output_folder, padding=5):
    """
    Function to apply Otsu's thresholding and keep only the largest object.
    Ensures files are saved in appropriate subfolders.
    Ignores pixels with intensity 0-5 for thresholding.
    """
    # Ensure the output directories exist
    os.makedirs(binary_output_folder, exist_ok=True)
    os.makedirs(grayscale_output_folder, exist_ok=True)

    subfolders = [f.path for f in os.scandir(input_folder) if f.is_dir()]

    for subfolder in subfolders:
        subfolder_name = os.path.basename(subfolder)

        # Prepare output subfolders
        binary_subfolder_output = os.path.join(binary_output_folder, subfolder_name)
        grayscale_subfolder_output = os.path.join(grayscale_output_folder, subfolder_name)

        # Create subdirectories if they don't exist
        os.makedirs(binary_subfolder_output, exist_ok=True)
        os.makedirs(grayscale_subfolder_output, exist_ok=True)

        image_files = [f for f in os.listdir(subfolder) if f.lower().endswith(('png', 'jpg', 'jpeg'))]

        print(f"Processing {len(image_files)} images in folder: {subfolder_name}")
        for filename in image_files:
            start_time = time.time()
            image_path = os.path.join(subfolder, filename)

            # Load the image in grayscale
            image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            if image is None:
                print(f"Warning: Could not load image {filename}. Skipping.")
                continue

            # Set pixels with intensity 0-5 to 0
            image = np.where(image <= 5, 0, image)


            # Add padding around the image
            padded_image = cv2.copyMakeBorder(image, padding, padding, padding, padding, cv2.BORDER_CONSTANT, value=0)

            # Apply Otsu's thresholding
            _, otsu_mask = cv2.threshold(padded_image.astype(np.uint8), 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

            # Invert binary mask if the background is white
            otsu_mask = cv2.bitwise_not(otsu_mask) if np.mean(otsu_mask) > 127 else otsu_mask

            # Detect contours
            contours, _ = cv2.findContours(otsu_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            # Keep only the largest contour
            if contours:
                largest_contour = max(contours, key=cv2.contourArea)

                # Create a mask with only the largest object
                largest_object_mask = np.zeros_like(otsu_mask)
                cv2.drawContours(largest_object_mask, [largest_contour], -1, 255, thickness=cv2.FILLED)

                # Binary output (Otsu with largest object only)
                binary_output_path = os.path.join(binary_subfolder_output, filename)
                cv2.imwrite(binary_output_path, largest_object_mask)

                # Grayscale output (Original image masked by largest object)
                grayscale_masked = cv2.bitwise_and(padded_image, padded_image, mask=largest_object_mask)
                grayscale_output_path = os.path.join(grayscale_subfolder_output, filename)
                cv2.imwrite(grayscale_output_path, grayscale_masked)

                print(f"Processed {filename} in {time.time() - start_time:.2f} seconds")
            else:
                print(f"No contours found for {filename}. Skipping.")

if __name__ == "__main__":
    input_directory = r"C:\Users\USER\Desktop\Selected Perogram\Preproses\Tahap B"
    binary_output_directory = r"C:\Users\USER\Desktop\Selected Perogram\Preproses\Otsu Threshold\Biner"
    grayscale_output_directory = r"C:\Users\USER\Desktop\Selected Perogram\Preproses\Otsu Threshold\Non Biner"

    print(f"Processing images from {input_directory}...")
    otsu_threshold_and_largest_object(
        input_directory,
        binary_output_directory,
        grayscale_output_directory,
        padding=5  # Adjust padding as needed
    )
    print(f"Processing complete. Results saved in {binary_output_directory} and {grayscale_output_directory}.")


k Means

In [ ]:
import cv2
import numpy as np
import os

def kmeans_segmentation(input_folder, binary_output_folder, grayscale_output_folder, k=2):
    """
    Performs K-Means segmentation on images in the input folder and subfolders,
    and saves binary and non-binary results in respective output folders.

    Parameters:
    - input_folder (str): Path to the folder containing input images.
    - binary_output_folder (str): Path to save the binary segmented images.
    - grayscale_output_folder (str): Path to save the non-binary segmented images.
    - k (int): Number of clusters for K-Means.
    """
    # Create main output directories if they don't exist
    os.makedirs(binary_output_folder, exist_ok=True)
    os.makedirs(grayscale_output_folder, exist_ok=True)

    # Walk through the input directory, including subfolders
    for root, _, files in os.walk(input_folder):
        for filename in files:
            if filename.lower().endswith(('png', 'jpg', 'jpeg')):
                # Construct full paths for input and output
                image_path = os.path.join(root, filename)
                rel_path = os.path.relpath(root, input_folder)  # Relative path to recreate folder structure
                binary_output_subfolder = os.path.join(binary_output_folder, rel_path)
                grayscale_output_subfolder = os.path.join(grayscale_output_folder, rel_path)

                # Ensure output subdirectories exist
                os.makedirs(binary_output_subfolder, exist_ok=True)
                os.makedirs(grayscale_output_subfolder, exist_ok=True)

                # Load the image
                image = cv2.imread(image_path)
                if image is None:
                    print(f"Warning: Could not load image {filename}. Skipping.")
                    continue

                print(f"Processing image: {image_path}")

                # Step 1: Convert image to RGB if grayscale
                if len(image.shape) == 2:
                    image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)

                # Step 2: Apply Smoothing (2D Convolution)
                kernel = np.ones((5, 5), np.float32) / 25
                smoothed_image = cv2.filter2D(image, -1, kernel)

                # Step 3: Reshape image for K-Means
                pixel_values = smoothed_image.reshape((-1, 3))
                pixel_values = np.float32(pixel_values)

                # Step 4: Define K-Means criteria and apply
                criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.2)
                _, labels, centers = cv2.kmeans(pixel_values, k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)

                # Step 5: Convert centers to uint8 and recreate segmented image
                centers = np.uint8(centers)
                segmented_image = centers[labels.flatten()]
                segmented_image = segmented_image.reshape(image.shape)

                # Step 6: Detect Contours on Segmented Image
                gray_segmented = cv2.cvtColor(segmented_image, cv2.COLOR_BGR2GRAY)
                _, binary_segmented = cv2.threshold(gray_segmented, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
                contours, _ = cv2.findContours(binary_segmented, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

                # Keep only the largest contour
                if contours:
                    largest_contour = max(contours, key=cv2.contourArea)
                    mask = np.zeros_like(binary_segmented)
                    cv2.drawContours(mask, [largest_contour], -1, 255, thickness=cv2.FILLED)
                    non_binary_result = cv2.bitwise_and(image, image, mask=mask)

                    # Save binary and non-binary results
                    binary_output_path = os.path.join(binary_output_subfolder, filename)
                    grayscale_output_path = os.path.join(grayscale_output_subfolder, filename)

                    cv2.imwrite(binary_output_path, mask)
                    cv2.imwrite(grayscale_output_path, non_binary_result)

                    print(f"Saved binary result to: {binary_output_path}")
                    print(f"Saved non-binary result to: {grayscale_output_path}")
                else:
                    print(f"No contours found for {filename}. Skipping.")

if __name__ == "__main__":
    input_directory = r"C:\Users\USER\Desktop\Selected Perogram\Preproses\Tahap B"
    binary_output_directory = r"C:\Users\USER\Desktop\Selected Perogram\Preproses\K Means\Biner"
    grayscale_output_directory = r"C:\Users\USER\Desktop\Selected Perogram\Preproses\K Means\Non Biner"

    print(f"Processing images from {input_directory}...")
    kmeans_segmentation(
        input_directory,
        binary_output_directory,
        grayscale_output_directory,
        k=2  # Number of clusters
    )
    print(f"Processing complete. Results saved in {binary_output_directory} and {grayscale_output_directory}.")
